The Linear Regression Model attempts to model the relationship between two variables by fitting a linear equation (a line) to observed data. In the model, one variable is considered to be an explanatory variable (X Variable), and the other is considered to be a dependent variable (Y Variable).

we are going to try to model the relationships between two financial assets, the price of a single share of Exxon Mobile stock and the price of a barrel of oil.<br>
 The question we are trying to answer is, does the explanatory variable (Oil) do a good job at predicting the dependent variable (a single share of Exxon Mobile stock.)<br>

explanatory variable (Oil) == (X Variable)
dependent variable (a single share of Exxon Mobile stock == (Y Variable)




In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import math 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from scipy import stats
from scipy.stats import kurtosis, skew

%matplotlib inline

# load the data

In [12]:
#load the data
price_data = pd.read_excel("oil_exxon.xlsx")

#set the date column as the index
price_data.index = pd.to_datetime(price_data['date'])
#drop the old date column
price_data = price_data.drop(['date'], axis=1)


price_data.head()

,exon_price,oil_price
date,,
2014-03-28,97.699997,106.64
2014-03-31,97.680000,105.95
2014-04-01,97.730003,105.70
2014-04-02,97.949997,103.37
2014-04-03,97.930000,104.88


# Clean the Data

In [14]:
#check the data types
price_data.dtypes

exon_price    float64
oil_price     float64
dtype: object

In [15]:
# column name is spelt incorrectly
# lets rename the columm name exon to exxon
new_column_names = {'exon_price':'exxon_price'}
price_data = price_data.rename(columns=new_column_names)
price_data.head()

,exxon_price,oil_price
date,,
2014-03-28,97.699997,106.64
2014-03-31,97.680000,105.95
2014-04-01,97.730003,105.70
2014-04-02,97.949997,103.37
2014-04-03,97.930000,104.88


#### check for missing values

In [19]:
display(price_data.isna().any())

#drop any missing values
price_data = price_data.dropna()
display(price_data.isna().any())

exxon_price    False
oil_price       True
dtype: bool

exxon_price    False
oil_price      False
dtype: bool